In [1]:
# 1. 8-퍼즐을 너비 우선 탐색으로 해결
from math import sqrt 
from queue import Queue

start='806524731'
goal ='123456780'

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(p):
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Move(p,i,i-1))
    if not (i%n==n-1): child.append(Move(p,i,i+1))
    if i>=n: child.append(Move(p,i,i-n))
    if i<n*(n-1): child.append(Move(p,i,i+n))
    return child

print_puzzle(start)

Q=Queue()
Q.put(start)
V=[start]
while not Q.empty():
    node=Q.get()
    if node==goal:
        print(len(V),"개 노드를 방문하고 답을 찾았다.")
        break
    else: 
        child=Branching(node)
        for j in range(len(child)):
            if child[j] not in V:
                Q.put(child[j])
                V.append(child[j])



806
524
731

169973 개 노드를 방문하고 답을 찾았다.


In [5]:
# 2. 8-퍼즐을 너비 우선 탐색으로 해결(경로 출력 기능 추가)
from math import sqrt
from queue import Queue

start='806524731'
goal='123456780'

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=Queue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V:
                Q.put(child[j])
                V.append(child[j].state)

print(f'{node.path} ({len(node.path)-1})')

806
524
731

169973 개 노드를 살피고 찾았다!
-DRULLDRDRULDLURULDDRUURDD (25)


In [21]:
# 번외 위에서 찾은 결과로 퍼즐 맞추기
from math import sqrt
from time import sleep
from IPython.display import clear_output
path = '-DRULLDRDRULDLURULDDRUURDD'
start = '806524731'

def Move_puzzle(s,i,direction):
    n=int(sqrt(len(s)))
    direction2idx ={'U':i-n,'D':i+n,'L':i-1,'R':i+1,'-':i}
    j = direction2idx[direction]
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def solve_puzzle(start,path,delay_time=3):
    puzzle = start
    for dir in path+'-':
        clear_output(wait=False)
        print(dir)
        print_puzzle(puzzle)
        puzzle = Move_puzzle(puzzle,puzzle.index('0'),dir)
        sleep(delay_time)

solve_puzzle(start,path)

-
123
456
780



In [9]:
# 3. 8-퍼즐을 최고 우선 탐색으로 해결(불일치 칸을 세는 평가 함수 사용)
from math import sqrt 
from queue import PriorityQueue

start='806524731'
goal='123456780'

n=int(sqrt(len(goal)))

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path
        self.val=self.f()

    def f_mismatch(self):
        return sum(self.state[i]!=goal[i] for i in range(len(goal)))
    
    def f_manhattan(self):
        dist=0
        for i in range(len(goal)):
            r,c=i//n,i%n
            r1,c1=self.state.index(goal[i])//n,self.state.index(goal[i]%n)
            dist += abs(r1-r)+abs(c1-c)
        return dist
    
    def f(self): 
        return self.f_mismatch()
    
    def __lt__(self,other):
        return self.val<other.val

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=PriorityQueue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V: 
                Q.put(child[j])
                V.append(node.state)
                
                
solve_puzzle(start,node.path)
print(f'{node.path} ({len(node.path)-1})')

806
524
731

1560 개 노드를 살피고 찾았다!
-DDRULDRUULDLURRDLDRUULDLDRUURDLDRUULDDLURURDLDRUULDDRUULDRULDLURRDLULDRRD (73)


In [22]:
# 4. 8-퍼즐을 최고 우선 탐색으로 해결(맨해튼 거리를 재는 평가 함수 사용)
from math import sqrt 
from queue import PriorityQueue

start='806524731'
goal='123456780'

n=int(sqrt(len(goal)))

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path
        self.val=self.f()

    def f_mismatch(self):
        return sum(self.state[i]!=goal[i] for i in range(len(goal)))
    
    def f_manhattan(self):
        dist=0
        for i in range(len(goal)):
            r,c=i//n,i%n
            r1,c1=self.state.index(goal[i])//n,self.state.index(goal[i])%n
            dist += abs(r1-r)+abs(c1-c)
        return dist
    
    def f(self): 
        return self.f_manhattan()
    
    def __lt__(self,other):
        return self.val<other.val

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=PriorityQueue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V: 
                Q.put(child[j])
                V.append(node.state)
                
                
solve_puzzle(start,node.path)
print(f'{node.path} ({len(node.path)-1})')

806
524
731

488 개 노드를 살피고 찾았다!
-DRDLURDLULURDDLURULDDRRUULDDLURRDLUURDDLLURRDLULDRR (51)


In [26]:
# 5. 8-퍼즐을 A*알고리즘으로 해결
from math import sqrt 
from queue import PriorityQueue

start='806524731'
goal='123456780'

n=int(sqrt(len(goal)))

class Node():
    def __init__(self,state,path):
        self.state=state
        self.path=path
        self.val=self.f()

    def f_mismatch(self):
        return sum(self.state[i]!=goal[i] for i in range(len(goal)))
    
    def f_manhattan(self):
        dist=0
        for i in range(len(goal)):
            r,c=i//n,i%n
            r1,c1=self.state.index(goal[i])//n,self.state.index(goal[i])%n
            dist += abs(r1-r)+abs(c1-c)
        return dist
    
    def f_A_star(self):
        return len(self.path)+self.f_manhattan()
    
    def f(self): 
        return self.f_A_star()
    
    def __lt__(self,other):
        return self.val<other.val

def print_puzzle(p):
    n=int(sqrt(len(p)))
    print('\n'.join([p[n*i:n*(i+1)] for i in range(n)]),end='\n\n')

def Move(s,i,j):
    if '@' in s: return None 
    return s.replace(s[i],'@').replace(s[j],s[i]).replace('@',s[j])

def Branching(node):
    p=node.state
    n=int(sqrt(len(p)))
    i=p.index('0')
    child=[]
    if not (i%n==0): child.append(Node(Move(p,i,i-1),node.path+'L'))
    if not (i%n==n-1): child.append(Node(Move(p,i,i+1),node.path+'R'))
    if i>=n: child.append(Node(Move(p,i,i-n),node.path+'U'))
    if i<n*(n-1): child.append(Node(Move(p,i,i+n),node.path+'D'))
    return child

print_puzzle(start)
Q=PriorityQueue()
root=Node(start,'-')
Q.put(root)
V=[root.state]
while not Q.empty():
    node=Q.get()
    if node.state==goal:
        print(len(V),'개 노드를 살피고 찾았다!')
        break
    else:
        child=Branching(node)
        for j in range(len(child)):
            if child[j].state not in V: 
                Q.put(child[j])
                V.append(node.state)
                
                
solve_puzzle(start,node.path)
print(f'{node.path} ({len(node.path)-1})')

-
123
456
780

-DRULLDRDRULDLURULDDRUURDD (25)


In [11]:
# 6. 틱택토를 플레이하는 미니맥스 알고리즘
n=3
start='-'*(n*n)

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  0123456789012345'[:n+2])
    for i in range(n):
        print(str(i%10)+':'+state[n*i:n*(i+1)])

def get_empty(state):
    if decide_winner(state) in ['O','X','T']:return []
    empty=[]
    for i in range(len(start)):
        if state[i]=='-':empty.append(i)
    return empty

def decide_winner(state):
    for (a,b,c) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
        if state[a]==state[b]==state[c]:
            if state[a]=='O': return 'O'
            elif state[a]=='X': return 'X'
    if [i for i in range(n*n) if state[i]=='-']==[]: return 'T'
    return 'N'

def minimax(state,player,depth):
    winner=decide_winner(state)
    if winner=='X': return 1, None
    elif winner=='O': return -1,None
    elif winner=='T': return 0,None
    
    e=get_empty(state)
    if depth%2==0:
        vmax,bestpos=-100,None
        for pos in e: 
            v,_=minimax(Move(state,pos,player),switch_player(player),depth+1)
            if v>vmax: vmax,bestpos=v,pos
        return vmax,bestpos
    else: 
        vmin,bestpos=100,None
        for pos in e:
            v,_=minimax(Move(state,pos,player),switch_player(player),depth+1)
            if v<vmin:vmin,bestpos=v,pos
        return vmin,bestpos
        
def tictactoe_play(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        if player=='X':
            val,pos=minimax(state,player,0)
        elif player=='O':
            x,y=input('사람 차례입니다. (x와 y를 공백 구분하여 입력하세요.)').split(' ')
            pos=int(y)*n+int(x)
            if state[pos]!='-': 
                print('둘 수 없는 곳입니다.')
                continue
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T':print('비겼습니다.')
            else: print(f'{winner}가 이겼습니다.')
            break
        player=switch_player(player)

tictactoe_play('O')

  012
0:---
1:---
2:---
  012
0:---
1:-O-
2:---
  012
0:X--
1:-O-
2:---
  012
0:X-O
1:-O-
2:---
  012
0:X-O
1:-O-
2:X--
  012
0:X-O
1:OO-
2:X--
  012
0:X-O
1:OOX
2:X--
  012
0:XOO
1:OOX
2:X--
  012
0:XOO
1:OOX
2:XX-
  012
0:XOO
1:OOX
2:XXO
비겼습니다.


In [2]:
# 7. 틱택토를 플레이하는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import random

n=3
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state = state
        self.player = player
        self.pos = pos
        self.parent = parent
        self.nwin = 0
        self.nvisit = 0
        self.untried=get_empty(state)
        self.children=[]

    def UCTselect(self):
        s=sorted(self.children, key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]

    def makeChild(self,state,pos,player):
        childnode=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(childnode)
        return childnode 
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T':self.nwin+=0.5
        elif winner==self.player:self.nwin+=1
    
    def __repr__(self):
        return str(self.state)+" "+str(self.nwin)+"/"+str(self.nvisit)

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  01234567890123456'[:n+2])
    for i in range(n): print(str(i%10)+':'+state[n*i:n*(i+1)])
    
def get_empty(state):
    if decide_winner(state) in ['O','X','T']: return []
    empty=[]
    for i in range(len(start)):
        if state[i]=='-':empty.append(i)
    return empty

def decide_winner(state):
    for (a,b,c) in[(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
        if state[a]==state[b]==state[c]:
            if state[a]=='O': return 'O'
            elif state[b]=='X': return 'X'
    if [i for i in range(n*n) if state[i]=='-']==[]: return 'T'
    return 'N' 

def mcts(state,player): 
    root=Node(state,player)

    for i in range(10000):
        node = root
        state = node.state
        roll_player = player
        while node.untried==[] and node.children!=[]:
            node = node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state = Move(state,pos,roll_player)
            node = node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]:break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player) 
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node = node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def tictactoe_play(first_mover):
    state = start
    player=first_mover
    print_board(state)
    while True:
        if player=='X':
            print("컴퓨터 차례입니다.")
            pos = mcts(state,player)
        elif player=='O':
            x,y=input("사람 차례입니다.(x와 y를 공백 구분하여 입력하세요.)").split()
            pos = int(y)*n+int(x)
            if state[pos]!='-':
                print("둘 수 없는 곳입니다.")
                continue
        state = Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T':print('비겼습니다.')
            else: print(winner,'가 이겼습니다.')
            break
        player=switch_player(player)

tictactoe_play('O')

  012
0:---
1:---
2:---


  012
0:---
1:-O-
2:---
컴퓨터 차례입니다.
  012
0:---
1:-O-
2:--X
  012
0:-O-
1:-O-
2:--X
컴퓨터 차례입니다.
  012
0:-O-
1:-O-
2:-XX
  012
0:-O-
1:-O-
2:OXX
컴퓨터 차례입니다.
  012
0:-OX
1:-O-
2:OXX
  012
0:-OX
1:-OO
2:OXX
컴퓨터 차례입니다.
  012
0:-OX
1:XOO
2:OXX
  012
0:OOX
1:XOO
2:OXX
비겼습니다.


In [5]:
# 8. 오목을 두는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repr__(self):
        return f'\n{board_ui(self.state)}\n {self.nwin}/{self.nvisit}  {self.nwin/self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def board_ui(state):
    return '  0123456789012345'[:n+2]+''.join([f'\n{i%10}:{state[n*i:n*(i+1)]}' for i in range(n)])

def print_board(state):
    print(board_ui(state))

def yx2pos(y,x):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def mcts(state,player):
    root = Node(state,player)
    
    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]: break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player)
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node=node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def omok_play(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        t1=time.time()
        if player=='X':
            if state==start: pos=randomAroundCenter(n)
            else: pos=mcts(state,player)
        elif player=='O':
            x,y=input("사람 차례입니다. (x와 y를 공백 구분하여 입력하세요.)").split()
            pos=yx2pos(int(y),int(x))
            print(pos)
            if state[pos]!='-':
                print('둘 수 없는 곳 입니다.')
                continue
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T': print('비겼습니다.')
            else : print(f'{winner}가 이겼습니다.')
            break
        print(f'{player}가 {round(time.time()-t1,3)}초를 썼습니다.')
        player=switch_player(player)

        
omok_play('O')


    

  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:----------
6:----------
7:----------
8:----------
9:----------
44
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----O-----
5:----------
6:----------
7:----------
8:----------
9:----------
O가 6.024초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----O-----
5:-----X----
6:----------
7:----------
8:----------
9:----------
X가 82.783초를 썼습니다.
53
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----O-----
5:---O-X----
6:----------
7:----------
8:----------
9:----------
O가 23.737초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:-----X----
4:----O-----
5:---O-X----
6:----------
7:----------
8:----------
9:----------
X가 81.394초를 썼습니다.
45
  0123456789
0:----------
1:----------
2:----------
3:-----X----
4:----OO----
5:---O-X----
6:----------
7:----------
8:----------
9:----------
O가 33.641초를 썼습니다.
  0123456789
0:----------
1:----------
2:---

In [6]:
# 9.자율 플레이하는 오목 인공지능
from math import sqrt,log
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repr__(self):
        return f'\n{board_ui(self.state)}\n {self.nwin}/{self.nvisit}  {self.nwin/self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def board_ui(state):
    return '  0123456789012345'[:n+2]+''.join([f'\n{i%10}:{state[n*i:n*(i+1)]}' for i in range(n)])

def print_board(state):
    print(board_ui(state))

def yx2pos(y,x):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def mcts(state,player):
    root = Node(state,player)
    
    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]: break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player)
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node=node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def omok_selfplay(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        t1=time.time()
        if state==start: pos=randomAroundCenter(n)
        else: pos=mcts(state,player)
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T': print('비겼습니다.')
            else : print(f'{winner}가 이겼습니다.')
            break
        print(f'{player}가 {round(time.time()-t1,3)}초를 썼습니다.')
        player=switch_player(player)

        
omok_selfplay('O')


    

  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:----------
6:----------
7:----------
8:----------
9:----------
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:------O---
6:----------
7:----------
8:----------
9:----------
O가 0.0초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X----
5:------O---
6:----------
7:----------
8:----------
9:----------
X가 81.653초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X-O--
5:------O---
6:----------
7:----------
8:----------
9:----------
O가 82.682초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X-O--
5:-----XO---
6:----------
7:----------
8:----------
9:----------
X가 81.121초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X-O--
5:-----XO---
6:-----O----
7:----------
8:----------
9:----------
O가 77.809초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:-

In [ ]:
# 연습문제 1 11.2.1절에서는 [그림 11-3]에 대한 [알고리즘 11-1]의 시뮬레이션을 부분적으로 제시한다. 나머지를 완성하시오.
# node=None Q=(1)               V=(1)
# node=1    Q=(2,3,4)           V=(1,2,3,4)
# node=2    Q=(3,4,6)           V=(1,2,3,4,6)
# node=3    Q=(4,6,8)           V=(1,2,3,4,6,8)
# node=4    Q=(6,8,9,10,11)     V=(1,2,3,4,6,8,9,10,11)
# node=6    Q=(8,9,10,11,13,14) V=(1,2,3,4,6,8,9,10,11,13,14)
# 위와 같은 방식으로 처리된다.

In [ ]:
# 연습문제 2 [그림 11-3]의 9,10,11노드에 대해 [그림 11-4]의 휴리스틱 함수를 계산하시오.
# 불일치 수 휴리스틱
# 9: 6 | 10: 7 | 11: 7
# 맨해튼 거리 휴리스틱 
# 9: 16 | 10: 14 | 11: 14

In [ ]:
# 연습문제 3 [프로그램 11-6]의 64행은 val을 받는데 쓰지 않는다. val의 쓸모를 구상하시오.
# val은 minimax 탐색하여 가장 확률이 높은 게임 결과를 알려준다.
# 이를 이용하여 게임 결과를 예측하는데 쓸 수 있다  

In [6]:
# 연습문제 4 [프로그램 11-6] 미니 맥스 알고리즘이 'OX-OX---O' 상태에서 어디에 두는지 확인하시오. 
# 의외의 수를 둔다면 이유를 설명 하고 해결책을 제시하시오.
# 02행에서 start='OX-OX---O'로 수정하고, 80행에서 tictactoe_play('X')를 호출하면 어디 두는지 확인할 수 있다.
# 6. 틱택토를 플레이하는 미니맥스 알고리즘
n=3
start='OX-OX---O'

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  0123456789012345'[:n+2])
    for i in range(n):
        print(str(i%10)+':'+state[n*i:n*(i+1)])

def get_empty(state):
    if decide_winner(state) in ['O','X','T']:return []
    empty=[]
    for i in range(len(start)):
        if state[i]=='-':empty.append(i)
    return empty

def decide_winner(state):
    for (a,b,c) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
        if state[a]==state[b]==state[c]:
            if state[a]=='O': return 'O'
            elif state[a]=='X': return 'X'
    if [i for i in range(n*n) if state[i]=='-']==[]: return 'T'
    return 'N'

def minimax(state,player,depth):
    winner=decide_winner(state)
    arr=[]
    if winner=='X': return 1, None,arr
    elif winner=='O': return -1,None,arr
    elif winner=='T': return 0,None,arr
    
    e=get_empty(state)
    if depth%2==0:
        vmax,bestpos=-100,None
        for pos in e: 
            v,_,_=minimax(Move(state,pos,player),switch_player(player),depth+1)
            arr.append((v,pos))
            if v>vmax: vmax,bestpos=v,pos
        return vmax,bestpos,arr
    else: 
        vmin,bestpos=100,None
        for pos in e:
            v,_,_=minimax(Move(state,pos,player),switch_player(player),depth+1)
            if v<vmin:vmin,bestpos=v,pos
        return vmin,bestpos,arr
        
def tictactoe_play(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        if player=='X':
            val,pos,arr=minimax(state,player,0)
            print(arr)
        elif player=='O':
            x,y=input('사람 차례입니다. (x와 y를 공백 구분하여 입력하세요.)').split(' ')
            pos=int(y)*n+int(x)
            if state[pos]!='-': 
                print('둘 수 없는 곳입니다.')
                continue
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T':print('비겼습니다.')
            else: print(f'{winner}가 이겼습니다.')
            break
        player=switch_player(player)
        break

tictactoe_play('X')

# minimax 알고리즘으로 수를 결정 할때 얼마나 빨리 이기는 지 보다는 이길 수 있는지 없는지가 중요하므로
# 승리가 이미 확정된 시점에서 2,0에 수를 두느냐 1,2에 수를 두느냐 는 중요하지 않게 된다.
# 그리고 수를 정할때 알고리즘 상으로 오름차순으로 정렬되어 수를 둘곳을 정하는데 
# 이때 2,0 수가 1,2보다 먼저 나오므로 해당 수를 채택하게 된다
# 이를 해결 하기위해서는 max depth를 함수에서 반환하여 max depth가 짧은 수를 채택하도록 하면 된다

  012
0:OX-
1:OX-
2:--O
[(-1, 2), (-1, 5), (1, 6), (1, 7)]
  012
0:OX-
1:OX-
2:X-O


In [15]:
# 연습문제 5 문제 5를 [프로그램 11-7]의 몬테카를로 트리 탐색 알고리즘에 적용하시오.
# 제대로 수를 둔다면 이유를 설명하시오.

# 7. 틱택토를 플레이하는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import random

n=3
start='OX-OX---O'

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state = state
        self.player = player
        self.pos = pos
        self.parent = parent
        self.nwin = 0
        self.nvisit = 0
        self.untried=get_empty(state)
        self.children=[]

    def UCTselect(self):
        s=sorted(self.children, key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]

    def makeChild(self,state,pos,player):
        childnode=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(childnode)
        return childnode 
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T':self.nwin+=0.5
        elif winner==self.player:self.nwin+=1
    
    def __repr__(self):
        return str(self.state)+" "+str(self.nwin)+"/"+str(self.nvisit)

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def print_board(state):
    print('  01234567890123456'[:n+2])
    for i in range(n): print(str(i%10)+':'+state[n*i:n*(i+1)])
    
def get_empty(state):
    if decide_winner(state) in ['O','X','T']: return []
    empty=[]
    for i in range(len(start)):
        if state[i]=='-':empty.append(i)
    return empty

def decide_winner(state):
    for (a,b,c) in[(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
        if state[a]==state[b]==state[c]:
            if state[a]=='O': return 'O'
            elif state[b]=='X': return 'X'
    if [i for i in range(n*n) if state[i]=='-']==[]: return 'T'
    return 'N' 

def mcts(state,player): 
    root=Node(state,player)

    for i in range(10000):
        node = root
        state = node.state
        roll_player = player
        while node.untried==[] and node.children!=[]:
            node = node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state = Move(state,pos,roll_player)
            node = node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]:break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player) 
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node = node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)

def tictactoe_play(first_mover):
    state = start
    player=first_mover
    print_board(state)
    while True:
        if player=='X':
            print("컴퓨터 차례입니다.")
            children = mcts(state,player)
            print(children)
            pos = children[-1].pos

        elif player=='O':
            x,y=input("사람 차례입니다.(x와 y를 공백 구분하여 입력하세요.)").split()
            pos = int(y)*n+int(x)
            if state[pos]!='-':
                print("둘 수 없는 곳입니다.")
                continue
        state = Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T':print('비겼습니다.')
            else: print(winner,'가 이겼습니다.')
            break
        player=switch_player(player)
        break

tictactoe_play('X')

# 몬테카를로 탐색 방법에서는 
# 2,0의 경우는 바로 이기는 것이 아니고 한개의 노드를 더 거치기 때문에 nwin/nvisit값이 더 떨어지게 된다.
# (몬테카를로 방법이므로 무조건 이기는 수가 아닌 랜덤한 수를 선택하게 된다.)
# 그러나 1,2의 경우는 바로 이기게 되므로 nwin/nvisit값이 1이 된다 (최댓값)
# 따라서 최선의 수를 선택 가능하다

  012
0:OX-
1:OX-
2:--O
컴퓨터 차례입니다.
[OXXOX---O 2/13, OX-OXX--O 2.5/13, OX-OX-X-O 4330.0/4354, OX-OX--XO 5620/5620]
  012
0:OX-
1:OX-
2:-XO
X 가 이겼습니다.


In [1]:
# 연습문제 6 [프로그램 11-8]에서 get_empty 함수는 8이웃을 사용해 후보 수를 모은다. 이를 24이웃을 사용하는 프로그램으로 확장하고,
# 계산 기간과 승리 확률 측면에서 원래 프로그램과 비교하시오.

# 8. 오목을 두는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repr__(self):
        return f'\n{board_ui(self.state)}\n {self.nwin}/{self.nvisit}  {self.nwin/self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def board_ui(state):
    return '  0123456789012345'[:n+2]+''.join([f'\n{i%10}:{state[n*i:n*(i+1)]}' for i in range(n)])

def print_board(state):
    print(board_ui(state))

def yx2pos(y,x):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-2,-1,0,1,2] for j in [-2,-1,0,1,2] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def mcts(state,player):
    root = Node(state,player)
    
    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]: break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player)
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node=node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def omok_selfplay(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        t1=time.time()
        if state==start: pos=randomAroundCenter(n)
        else: pos=mcts(state,player)
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T': print('비겼습니다.')
            else : print(f'{winner}가 이겼습니다.')
            break
        print(f'{player}가 {round(time.time()-t1,3)}초를 썼습니다.')
        player=switch_player(player)

        
omok_selfplay('O')

# 시간은 2~3배정도 더 걸리며 정확도는 약간더 올라갔다.
# 그러나 아직 한칸 떨어진 칸은 인식하고 방어하지 못하여 정확도가 떨어졌다

  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:----------
6:----------
7:----------
8:----------
9:----------
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:-----O----
6:----------
7:----------
8:----------
9:----------
O가 0.0초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:-----O----
6:---X------
7:----------
8:----------
9:----------
X가 216.413초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:-----O----
6:---XO-----
7:----------
8:----------
9:----------
O가 214.239초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:-------X--
4:----------
5:-----O----
6:---XO-----
7:----------
8:----------
9:----------
X가 209.625초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:-------X--
4:----------
5:-----O----
6:---XO-----
7:---O------
8:----------
9:----------
O가 215.955초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------

In [3]:
# 연습문제 7 [프로그램 11-9]는 두 복사본이 한번만 겨루고 끝난다.
# 이를 두복사본이 여러 번 겨루게 하고 각각의 승패 기록과 승률을 측정하는 프로그램으로 확장 하시오. 

# 9.자율 플레이하는 오목 인공지능
from math import sqrt,log
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repr__(self):
        return f'\n{board_ui(self.state)}\n {self.nwin}/{self.nvisit}  {self.nwin/self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def board_ui(state):
    return '  0123456789012345'[:n+2]+''.join([f'\n{i%10}:{state[n*i:n*(i+1)]}' for i in range(n)])

def print_board(state):
    print(board_ui(state))

def yx2pos(y,x):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def mcts(state,player):
    root = Node(state,player)
    
    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]: break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player)
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node=node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def omok_selfplay(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        t1=time.time()
        if state==start: pos=randomAroundCenter(n)
        else: pos=mcts(state,player)
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        print(f'{player}가 {round(time.time()-t1,3)}초를 썼습니다.')
        if winner!='N':return winner
        player=switch_player(player)

record = []        
for i in range(10):
    winner = omok_selfplay('O')
    print(i+1,'번 째 판은 ')
    if winner=='T': print('비겼습니다')
    else: print(winner,'가 이겼습니다.')
    record.append(winner)

print(record)
print(f'O의 승률은 {record.count("O")/len(record)} 승리 횟수: {record.count("O")}')
print(f'X의 승률은 {record.count("X")/len(record)} 승리 횟수: {record.count("X")}')

  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:----------
6:----------
7:----------
8:----------
9:----------
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----O----
5:----------
6:----------
7:----------
8:----------
9:----------
O가 0.0초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----OX---
5:----------
6:----------
7:----------
8:----------
9:----------
X가 83.89초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----O-----
4:-----OX---
5:----------
6:----------
7:----------
8:----------
9:----------
O가 83.014초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----O-----
4:-----OX---
5:------X---
6:----------
7:----------
8:----------
9:----------
X가 80.502초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----O-----
4:-----OX---
5:-----OX---
6:----------
7:----------
8:----------
9:----------
O가 80.822초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:--

In [6]:
# 연습문제 8 [프로그램 11-8]의 128행에서는 승률, 즉 nwin/nvisit가 최고인 자식 노드를 선택한다.
# 한 복사본은 [프로그램 11-8]처럼 승률이 최고인 자식 노드를 선택하고 다른 복사본은 방문 횟수,
# 즉 nvisit이 최고인 자식 노드를 선택하게하고 둘이 여러 번 겨루게 하여 누가 승률이 높은지 평가하시오.

# 9.자율 플레이하는 오목 인공지능
from math import sqrt,log
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repr__(self):
        return f'\n{board_ui(self.state)}\n {self.nwin}/{self.nvisit}  {self.nwin/self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def board_ui(state):
    return '  0123456789012345'[:n+2]+''.join([f'\n{i%10}:{state[n*i:n*(i+1)]}' for i in range(n)])

def print_board(state):
    print(board_ui(state))

def yx2pos(y,x):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def mcts(state,player,key=lambda c: c.nwin/c.nvisit):
    root = Node(state,player)
    
    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]: break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player)
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node=node.parent
    return sorted(root.children,key=key)[-1].pos

def omok_selfplay(first_mover):
    state=start
    player=first_mover
    print_board(state)

    X_key = lambda c:c.nwin/c.nvisit
    O_key = lambda c: c.nvisit
    while True:
        t1=time.time()
        if state==start: pos=randomAroundCenter(n)
        else: pos=mcts(state,player,key= O_key if player=='O'else X_key)
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T': print('비겼습니다.')
            else : print(f'{winner}가 이겼습니다.')
            break
        print(f'{player}가 {round(time.time()-t1,3)}초를 썼습니다.')
        player=switch_player(player)

        
omok_selfplay('O')


    

  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:----------
6:----------
7:----------
8:----------
9:----------
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:-----O----
6:----------
7:----------
8:----------
9:----------
O가 0.0초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X----
5:-----O----
6:----------
7:----------
8:----------
9:----------
X가 84.186초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X----
5:-----OO---
6:----------
7:----------
8:----------
9:----------
O가 82.556초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----X----
5:----XOO---
6:----------
7:----------
8:----------
9:----------
X가 80.947초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----XO---
5:----XOO---
6:----------
7:----------
8:----------
9:----------
O가 80.255초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:-

In [11]:
# 연습문제 9 [프로그램 11-8]의 순수 몬테카를로 트리 탐색 알고리즘은 가끔 상대가 [그림 11-11]의 파란 점에 두어 이기는 긴급 상황을 놓치는 경우가 발생한다.
# 긴급 상황을 알아낸는 critical_situation 함수를 추가해 프로그램을 개선하시오.
# 8. 오목을 두는 몬테카를로 트리 탐색 알고리즘
from math import sqrt,log
import numpy as np
import random 
import time 
n=10
k=5
start='-'*(n*n)

class Node():
    def __init__(self,state,player=None,pos=None,parent=None):
        self.state=state
        self.player=player
        self.pos=pos
        self.parent=parent
        self.nwin=0
        self.nvisit=0
        self.untried=get_empty(state)
        self.children=[]
        
    def UCTselect(self):
        s=sorted(self.children,key=lambda c: c.nwin/c.nvisit+sqrt(log(self.nvisit)/c.nvisit))
        return s[-1]
    
    def makeChild(self,state,pos,player):
        node=Node(state,player,pos,parent=self)
        self.untried.remove(pos)
        self.children.append(node)
        return node
    
    def update(self,winner):
        self.nvisit+=1
        if winner=='T': self.nwin+=0.5
        elif winner==self.player: self.nwin+=1
    
    def __repr__(self):
        return f'\n{board_ui(self.state)}\n {self.nwin}/{self.nvisit}  {self.nwin/self.nvisit}'

def randomAroundCenter(n):
    c=n*(n//2)+n//2
    return random.choice([c,c-1,c+1,c-n,c+n])

def Move(state,pos,player):
    return state[:pos]+player+state[pos+1:]

def switch_player(player):
    return 'X' if player=='O' else 'O'

def board_ui(state):
    return '  0123456789012345'[:n+2]+''.join([f'\n{i%10}:{state[n*i:n*(i+1)]}' for i in range(n)])

def print_board(state):
    print(board_ui(state))

def yx2pos(y,x):return y*n+x
def pos2yx(pos):return pos//n,pos%n

def get_empty(state):
    if decide_winner(state) in['O','X','T']: return []

    empty=[]
    for i in range(len(start)):
        if state[i]=='-': continue
        r,c=pos2yx(i)
        for (y,x) in [(r+i,c+j) for i in [-1,0,1] for j in [-1,0,1] if i!=0 or j!=0]:
            pos = y*n+x
            if 0<=x<n and 0<=y<n and state[pos]=='-' and pos not in empty: empty.append(pos)
    return empty
        
def decide_winner(state):
    nvoid = 0
    for i in range(n*n):
        s=state[i]
        if s=='-':
            nvoid+=1
            continue
        
        r,c=pos2yx(i)

        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]

        while c1>=0 and state[yx2pos(r,c1)]==s:c1-=1
        while c2<n and state[yx2pos(r,c2)]==s:c2+=1
        if c2-c1-1==k: return s

        while r1>=0 and state[yx2pos(r1,c)]==s:r1-=1
        while r2<n and state[yx2pos(r2,c)]==s:r2+=1
        if r2-r1-1==k: return s 
        
        (r1,c1),(r2,c2) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1>=0 and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1-1
        while r2<n and c2<n and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2+1
        if r2-r1-1==k: return s 

        (r1,c2),(r2,c1) = [(r+i,c+i) for i in [-1,1]]
        while r1>=0 and c1<n and state[yx2pos(r1,c1)]==s:r1,c1=r1-1,c1+1
        while r2<n and c2>=0 and state[yx2pos(r2,c2)]==s: r2,c2=r2+1,c2-1
        if r2-r1-1==k: return s 

    if nvoid==0: return 'T'
    return 'N'

def critical_situation(state, player):
    p=switch_player(player)
    for i in range(n*n):
        s=state[i]
        if s!='-': continue
        r,c=pos2yx(i)
        dots=np.array([0,0,0,0])
        for j in range(-2,3):
            if 0<=(r+j)<n and state[yx2pos(r+j,c)]==p:dots[0]+=1
            if 0<=(c+j)<n and state[yx2pos(r,c+j)]==p:dots[1]+=1
            if 0<=(r+j)<n and 0<=(c+j)<n and state[yx2pos(r+j,c+j)]==p:dots[2]+=1
            if 0<=(r-j)<n and 0<=(c+j)<n and state[yx2pos(r-j,c+j)]==p:dots[3]+=1
        if len(dots[np.where(dots>=k-2)]): return i
        elif len(dots[np.where(dots>=k-3)])>=2: return i
    return None

def mcts(state,player):
    root = Node(state,player)

    pos = critical_situation(state,player)
    if pos!=None: 
        print('critical_situation')
        return pos

    for i in range(10000):
        node=root
        state=node.state
        roll_player=player
        while node.untried==[] and node.children!=[]:
            node=node.UCTselect()
            state=Move(state,node.pos,roll_player)
            roll_player=switch_player(roll_player)
        
        if node.untried!=[]:
            pos=random.choice(node.untried)
            state=Move(state,pos,roll_player)
            node=node.makeChild(state,pos,roll_player)
            roll_player=switch_player(roll_player)
            
        while True:
            e=get_empty(state)
            if e==[]: break
            state=Move(state,random.choice(e),roll_player)
            roll_player=switch_player(roll_player)
        
        winner=decide_winner(state)
        while node!=None:
            node.update(winner)
            node=node.parent
    return sorted(root.children,key=lambda c: c.nwin/c.nvisit)[-1].pos

def omok_play(first_mover):
    state=start
    player=first_mover
    print_board(state)
    while True:
        t1=time.time()
        if state==start: pos=randomAroundCenter(n)
        else:pos=mcts(state,player)
        # if player=='X':
        #     if state==start: pos=randomAroundCenter(n)
        #     else: pos=mcts(state,player)
        # elif player=='O':
        #     x,y=input("사람 차례입니다. (x와 y를 공백 구분하여 입력하세요.)").split()
        #     pos=yx2pos(int(y),int(x))
        #     print(pos)
        #     if state[pos]!='-':
        #         print('둘 수 없는 곳 입니다.')
        #         continue
        state=Move(state,pos,player)
        print_board(state)
        winner=decide_winner(state)
        if winner in ['O','X','T']:
            if winner=='T': print('비겼습니다.')
            else : print(f'{winner}가 이겼습니다.')
            break
        print(f'{player}가 {round(time.time()-t1,3)}초를 썼습니다.')
        player=switch_player(player)

        
omok_play('O')


    

  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:----------
6:----------
7:----------
8:----------
9:----------
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:------O---
6:----------
7:----------
8:----------
9:----------
O가 0.0초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:----------
5:------O---
6:-----X----
7:----------
8:----------
9:----------
X가 81.891초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----O----
5:------O---
6:-----X----
7:----------
8:----------
9:----------
O가 82.715초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----------
4:-----O----
5:----X-O---
6:-----X----
7:----------
8:----------
9:----------
X가 82.18초를 썼습니다.
  0123456789
0:----------
1:----------
2:----------
3:----O-----
4:-----O----
5:----X-O---
6:-----X----
7:----------
8:----------
9:----------
O가 79.761초를 썼습니다.
  0123456789
0:----------
1:----------
2:---X------
3:--